In [6]:
import sglang as sgl

@sgl.function
def harry_potter_gen(s, name):
    s += sgl.user(f"Using JSON, describe the character {name} from Harry Potter.")
    s += sgl.assistant(sgl.gen("json", max_tokens=256))

sgl.set_default_backend(sgl.RuntimeEndpoint("http://localhost:30000"))
state = harry_potter_gen.run("Hermione Granger")
print(state["json"])
# >>>   Sure. Here's the character description based on JSON:
# >>>
# >>> ```json
# >>> {
# >>>   "name": "Hermione Granger",
# >>>   "age": 15,
# >>>   "species": "Human",
# >>>   "nationality": "British",
# >>>   "gender": "Female",
# >>>   "house": "Hufflepuff",
# >>>   "personality": "Intelligent, bookish, and compassionate",
# >>>   "status": "Alive",
# >>>   "birth_date": "1991-01-03",
# >>>   "death_date": null
# >>> }
# >>> ```

  Sure. Here's the character description based on JSON:

```json
{
  "name": "Hermione Granger",
  "age": 15,
  "species": "Human",
  "fictional_status": true,
  "nationality": "British",
  "gender": "Female",
  "house": "Hufflepuff",
  "looking_after": null,
  "mbti": "INFP",
  "personality": "Intelligent, bookish, independent, curious, and compassionate",
  "status": "Alive",
  "birth_date": "1991-01-03",
  "death_date": null
}
```

Please note that this character description is based on the character's portrayal in the Harry Potter series of books and movies.


In [1]:
import sglang as sgl

character_regex = (
    r"""\{
    "name": "[\w\d\s]{1,16}",
    "house": "(Gryffindor|Slytherin|Ravenclaw|Hufflepuff)",
    "blood status": "(Pure-blood|Half-blood|Muggle-born)",
    "wand": \{
        "wood": "[\w\d\s]{1,16}",
        "core": "[\w\d\s]{1,16}",
        "length": [0-9]{1,2}\.[0-9]{0,2}
    \}
\}"""
)

@sgl.function
def harry_potter_gen(s, name):
    s += sgl.user(f"Please describe the character {name} from Harry Potter.")
    s += sgl.assistant(sgl.gen("json", max_tokens=256, regex=character_regex))

sgl.set_default_backend(sgl.RuntimeEndpoint("http://localhost:30000"))
state = harry_potter_gen.run("Hermione Granger")
character_json = state["json"]

In [2]:
print(character_json)

{
    "name": "Hermione Granger",
    "house": "Hufflepuff",
    "blood status": "Pure-blood",
    "wand": {
        "wood": "Bludger",
        "core": "Phoenix",
        "length": 13.5
    }
}


In [37]:
import sglang as sgl
from sglang.srt.constrained import build_regex_from_object
from pydantic import BaseModel

class Wand(BaseModel):
    wood: str
    core: str
    length: float

class Character(BaseModel):
    name: str
    house: str
    blood_status: str
    wand: Wand

@sgl.function
def harry_potter_gen(s, name):
    s += sgl.user(f"Please describe the character {name} from Harry Potter.")
    s += sgl.assistant(
        sgl.gen("json", max_tokens=256, regex=build_regex_from_object(Character))
    )

sgl.set_default_backend(sgl.RuntimeEndpoint("http://localhost:30000"))
state = harry_potter_gen.run("Hermione Granger")
character_json = state["json"]

In [38]:
print(character_json)  # does not work

{"name"  : "Hermione Granger",se"use" : "blood_status"_statusand"   
                   ,d"and"  :  {"ore"":"-","h"re":"2nd"  , "length":4}  }


In [5]:
import sglang as sgl

@sgl.function
def harry_potter_gen(s, name):
    s += sgl.user(f"Please describe the character {name} from Harry Potter.")
    s += sgl.assistant('''{
    "name": "''' + sgl.gen("name", max_tokens=32, stop='"') + '''",
    "house": "''' + sgl.gen(
        "house", choices=["Gryffindor", "Slytherin", "Ravenclaw", "Hufflepuff"]
    ) + '''",
    "blood status": "''' + sgl.gen(
        "blood status", choices=["Pure-blood", "Half-blood", "Muggle-born"]
    ) + '''",
    "wand": {
        "wood": "''' + sgl.gen("wood", regex=r"[\w\d\s]{1,16}") + '''",
        "core": "''' + sgl.gen("core", regex=r"[\w\d\s]{1,16}") + '''",
        "length": ''' + sgl.gen("length", regex=r"[0-9]{1,2}\.[0-9]{0,2}") + '''
    }
}''')

sgl.set_default_backend(sgl.RuntimeEndpoint("http://localhost:30000"))
state = harry_potter_gen.run("Hermione Granger")
character_json = state.messages()[1]["content"]

In [6]:
print(character_json)

{
    "name": "Hermione Granger",
    "house": "Hufflepuff",
    "blood status": "Pure-blood",
    "wand": {
        "wood": "Oak treewood with an emerald core and dragon heartwood trim and grip with dragon",
        "core": "Eldritch Wood from a Goxdrich willow tree in the Forbidden Forest of",
        "length": 11.5136738863437
    }
}


In [7]:
import sglang as sgl

@sgl.function
def which_country_upper(s, city):
    s += sgl.user(f"In which country is {city} located?")
    s += sgl.assistant(sgl.gen("country", choices=["France", "Spain", "Italy"]))

@sgl.function
def which_country_lower(s, city):
    s += sgl.user(f"In which country is {city} located?")
    s += sgl.assistant(sgl.gen("country", choices=["france", "spain", "italy"]))

sgl.set_default_backend(sgl.RuntimeEndpoint("http://localhost:30000"))
state_upper = which_country_upper.run("Paris")
state_lower = which_country_lower.run("Paris")
print(state_upper["country"])  # >>> France
print(state_lower["country"])  # >>> spain

France
spain


In [8]:
import sglang as sgl

@sgl.function
def us_president_choices(s):
    s += sgl.user("Name a US president.")
    s += sgl.assistant(
        "An example of a US president is " +
        sgl.gen("president", choices=["Donald Duck", "Millard Fillmore"])
    )

@sgl.function
def us_president_regex(s):
    s += sgl.user("Name a US president.")
    s += sgl.assistant(
        "An example of a US president is " +
        sgl.gen("president", regex=r"(Donald Duck|Millard Fillmore)")
    )

sgl.set_default_backend(sgl.RuntimeEndpoint("http://localhost:30000"))
state_choices = us_president_choices.run()
state_regex = us_president_regex.run()
print(state_choices["president"])  # >>> Millard Fillmore
print(state_regex["president"])  # >>> Donald Duck

Millard Fillmore
Donald Duck


In [35]:
import sglang as sgl

@sgl.function
def harry_potter_gen(s, names, max_characters=5):
    s += sgl.user(f"Describe these Harry Potter characters: {', '.join(names)}")
    n = 1
    while n <= max_characters:
        s += sgl.assistant('''{
    "name": "''' + sgl.gen("name", max_tokens=32, stop='"') + '''",
    "house": "''' + sgl.gen(
        "house", choices=["Gryffindor", "Slytherin", "Ravenclaw", "Hufflepuff"]
    ) + '''",
    "blood status": "''' + sgl.gen(
        "blood status", choices=["Pure-blood", "Half-blood", "Muggle-born"]
    ) + '''",
    "wand": {
        "wood": "''' + sgl.gen("wood", max_tokens=32, stop='"') + '''",
        "core": "''' + sgl.gen("core", max_tokens=32, stop='"') + '''",
        "length": ''' + sgl.gen("length", regex=r"[0-9]{1,2}\.[0-9]{0,2}") + '''
    }
}''')
        s += sgl.user("Are there any more characters to describe? (Y/N)")
        s += sgl.assistant(sgl.gen(f"continue_{n}", choices=["Y", "N"]))
        if s[f"continue_{n}"] == "N":
            break
        n += 1
        s += sgl.user(f"OK, describe the next character.")
    s["n"] = min(n, max_characters)

sgl.set_default_backend(sgl.RuntimeEndpoint("http://localhost:30000"))
state = harry_potter_gen.run(["Ron Weasley", "Snape"])
characters_json = [state.messages()[1+i*4]["content"] for i in range(state["n"])]

In [36]:
for character in characters_json:
    print(character)
    print()

{
    "name": "Ron Weasley",
    "house": "Hufflepuff",
    "blood status": "Pure-blood",
    "wand": {
        "wood": "Flint",
        "core": "Dragonheart",
        "length": 11.50
    }
}

{
    "name": "Snape",
    "house": "Slytherin",
    "blood status": "Pure-blood",
    "wand": {
        "wood": "Ash",
        "core": "Phoenix tail",
        "length": 13.50
    }
}



In [12]:
import sglang as sgl

@sgl.function
def grass(s):
    s += sgl.user(f"What colour is grass?")
    s += sgl.assistant(sgl.gen("colour", regex=f"(Grey|Forest green)"))

colours = {"Grey": "Grey", "Forest green": "Forest green", "Green": "Forest green"}

@sgl.function
def grass_aliases(s):
    s += sgl.user(f"What colour is grass?")
    s += sgl.assistant(sgl.gen("colour", regex=f"(Grey|Forest green|Green)"))

sgl.set_default_backend(sgl.RuntimeEndpoint("http://localhost:30000"))
state = grass.run()
state_aliases = grass_aliases.run()
print(state["colour"])  # >>> Grey
print(colours[state_aliases["colour"]])  # >>> Forest green

Grey
Forest green
